<a href="https://colab.research.google.com/github/Bharathbrothers/stark-notes/blob/master/tensorflow_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sklearn

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


TensorFlow 2.x selected.


In [0]:
import io
from google.colab import files
uploaded = files.upload()
df= pd.read_csv(io.BytesIO(uploaded['heart.csv']))


Saving heart.csv to heart.csv


In [0]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [0]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(df, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [0]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([46 45 65 53 66], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([0 1 1 1 0], shape=(5,), dtype=int32)


In [0]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [0]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [0]:
def numeric_column(key,
                   shape=(1,),
                   default_value=None,
                   dtype=np.dtype('float32'),
                   normalizer_fn=None):
    shape = _check_shape(shape, key)
    if not (dtype.is_integer or dtype.is_floating):
        raise ValueError('dtype must be convertible to float. '
                        'dtype: {}, key: {}'.format(dtype, key))
    default_value = fc_utils.check_default_value(
        shape, default_value, dtype, key)

    if normalizer_fn is not None and not callable(normalizer_fn):
        raise TypeError(
            'normalizer_fn must be a callable. Given: {}'.format(normalizer_fn))

    fc_utils.assert_key_is_string(key)
    return NumericColumn(
        key,
        shape=shape,
        default_value=default_value,
        dtype=dtype,
        normalizer_fn=normalizer_fn)

In [0]:
age = numeric_column("age")
demo(age)

NameError: ignored

In [0]:
feature_columns = []
feature_layer_inputs = {}

# numeric cols
for header in ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']:
  feature_columns.append(feature_column.numeric_column(header))

print(feature_columns)
feature_layer_inputs[header] = tf.keras.Input(shape=(1,), name=header)
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

[NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='sex', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='cp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='trestbps', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='chol', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='fbs', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='restecg', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='thalach', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='exang', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='oldpeak', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='s

In [0]:
batch_size = 45
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [0]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Train for 7 steps, validate for 2 steps
Epoch 1/5
7/7 [==============================] - 1s 131ms/step - loss: 1.1419 - accuracy: 0.6528 - val_loss: 3.7276 - val_accuracy: 0.5306
Epoch 2/5
7/7 [==============================] - 0s 8ms/step - loss: 2.5937 - accuracy: 0.5440 - val_loss: 1.6035 - val_accuracy: 0.5918
Epoch 3/5
7/7 [==============================] - 0s 8ms/step - loss: 0.8098 - accuracy: 0.6477 - val_loss: 1.2577 - val_accuracy: 0.5918
Epoch 4/5
7/7 [==============================] - 0s 8ms/step - loss: 0.5983 - accuracy: 0.7461 - val_loss: 0.8713 - val_accuracy: 0.6531
Epoch 5/5
7/7 [==============================] - 0s 8ms/step - loss: 0.8174 - accuracy: 0.6528 - val_loss: 3.7218 - val_accuracy: 0.4694


In [0]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 73ms/step - loss: 2.9443 - accuracy: 0.5574
Accuracy 0.55737704


In [0]:
model.save('tensor_classifier')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: tensor_classifier/assets


In [0]:
tf.saved_model.save(model,'tensor_classifier')

INFO:tensorflow:Assets written to: tensor_classifier/assets


In [0]:
converter = tf.lite.TFLiteConverter.from_saved_model('tensor_classifier')
tflite_model = converter.convert()

In [0]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

78876

In [0]:
try:
  from google.colab import files
  files.download(tflite_model_file)
except:
  pass

In [0]:
new_model = tf.models.load_model(tflite_model_file)
new_model.summary()

AttributeError: ignored